In [1]:
#--------------------Data Background--------------------#

print('Running Data Background')
import pandas as pd

# File paths
raw_data_path = '../raw data/'
cln_data_path = '../output/Data Background/'

treatmentlongnames = ['1seller_current', '2seller_current', '1seller_pricegrid_current', '2seller_pricegrid_current']
treatmentlist      = ['1s', '2s', '1g', '2g']
sessionlist        = ['session' + str(x) + '.xlsx' for x in range(1,15)]

raw_files      = [raw_data_path + x for x in sessionlist]
cln_files      = [cln_data_path + 'df_' + x + '.xlsx' for x in treatmentlist]
num_sessions   = len(raw_files)
num_treatments = len(cln_files)

# Combine session data into one file with multiple sheets
print('Combining data...')
combined_data_path = raw_data_path + 'combined_data.xlsx'
with pd.ExcelWriter(combined_data_path) as writer:
    for i, file in enumerate(raw_files):
        df = pd.read_excel(file)
        df.to_excel(writer, sheet_name=f'Sheet{i+1}', index=False)

# Make dfs for each treatment
print('Separating treatments...')
treatment_lists = [[] for x in range(num_treatments)]
for i in range(num_sessions):
    df = pd.read_excel(combined_data_path, sheet_name=i, header=None)
    df = df[df[2].isin(['subjects', 'treatments'])]
    df_treatmentIDs = df[df[1] == -1]
    df_treatmentIDs = df_treatmentIDs[1:]
    order = int(df_treatmentIDs[6].iloc[1] == '1seller_current')
    treatmentIDdict = {key: value for key, value in zip(df_treatmentIDs[3],df_treatmentIDs[6])}
    df[1] = df[1].map(treatmentIDdict)
    df = df[df[2] == 'subjects']
    dfs = [df[df[1] == x] for x in treatmentlongnames]
    
    for j in range(num_treatments):
        dfs[j].columns = dfs[j].iloc[0]
        dfs[j] = dfs[j][dfs[j]['Period'] != 'Period']
        dfs[j]['session'] = i + 1
        dfs[j]['order'] = order
        dfs[j] = dfs[j].drop(dfs[j].columns[0], axis=1)
        treatment_lists[j].append(dfs[j])

# Save treatment dfs
print('Saving data by treatment...')
cols_to_keep_1s = ['session', 'order', 'Subject', 'Group', 'is_seller', 'price_high_quality', 'price_low_quality', 'Period', 'buy_high_priced', 'buy_low_priced']
cols_to_keep_2s = ['session', 'order', 'Subject', 'Group', 'is_seller', 'price_high_quality', 'price_low_quality', 'Period', 'buy_hh', 'buy_ll', 'buy_hl']
cols_to_keep_1g = ['session', 'order', 'Subject', 'Group', 'is_seller', 'price_high_quality', 'price_low_quality', 'price_level', 'buy']
cols_to_keep_2g = ['session', 'order', 'Subject', 'Group', 'is_seller', 'price_high_quality', 'price_low_quality', 'price_level', 'buy']
cols_to_keep    = [cols_to_keep_1s, cols_to_keep_2s, cols_to_keep_1g, cols_to_keep_2g]

for j in range(num_treatments):
    df = pd.concat(treatment_lists[j], ignore_index=True)
    df = df[cols_to_keep[j]]
    df.to_excel(cln_files[j], index=False)
    
# Rename some stuff
df_1s = pd.read_excel(cln_data_path + 'df_1s.xlsx')
df_1s = df_1s.rename(columns={'price_high_quality': 'pshh', 'price_low_quality': 'pshl', 'buy_high_priced': 'pbhh', 'buy_low_priced': 'pbll'})
df_1s.to_excel(cln_data_path + 'df_1s.xlsx', index=False)

df_2s = pd.read_excel(cln_data_path + 'df_2s.xlsx')
df_2s['buy_h_hl'] = (df_2s['buy_hl'] == 2).astype(float)
df_2s['buy_l_hl'] = (df_2s['buy_hl'] == 1).astype(float)
df_2s = df_2s.rename(columns={'price_high_quality': 'pshh', 'price_low_quality': 'pshl', 'buy_hh': 'pbhh', 'buy_ll': 'pbll', 'buy_h_hl': 'pbhb', 'buy_l_hl': 'pblb'})
df_2s.to_excel(cln_data_path + 'df_2s.xlsx', index=False)

Running Data Background
Combining data...


KeyboardInterrupt: 

In [11]:
#--------------------Collapsing Data--------------------#

print('Aggregating by subject, round, and session...')
cln_data_path = '../output/Data Background/'

# Make dfs for just first treatments each session
df_1s = pd.read_excel(cln_data_path + 'df_1s.xlsx')
df_2s = pd.read_excel(cln_data_path + 'df_2s.xlsx')
df_1g = pd.read_excel(cln_data_path + 'df_1g.xlsx')
df_2g = pd.read_excel(cln_data_path + 'df_2g.xlsx')

df_1s_first = df_1s[df_1s['order'] == 1]
df_2s_first = df_2s[df_2s['order'] == 0]
df_1g_first = df_1g[df_1g['order'] == 1]
df_2g_first = df_2g[df_2g['order'] == 0]

df_1s_first.to_excel(cln_data_path + 'df_1s_first.xlsx', index=False)
df_2s_first.to_excel(cln_data_path + 'df_2s_first.xlsx', index=False)
df_1g_first.to_excel(cln_data_path + 'df_1g_first.xlsx', index=False)
df_2g_first.to_excel(cln_data_path + 'df_2g_first.xlsx', index=False)

# Aggregating data by subject, round, and session
df_1s_sel = df_1s[df_1s['is_seller'] == 1]
df_2s_sel = df_2s[df_2s['is_seller'] == 1]
df_1s_buy = df_1s[df_1s['is_seller'] == 0]
df_2s_buy = df_2s[df_2s['is_seller'] == 0]

bysubj_sel_1s = df_1s_sel.groupby(['session', 'Subject'])[['pshh', 'pshl']].mean()
bysubj_sel_2s = df_2s_sel.groupby(['session', 'Subject'])[['pshh', 'pshl']].mean()
bysubj_buy_1s = df_1s_buy.groupby(['session', 'Subject'])[['pbhh', 'pbll']].mean()
bysubj_buy_2s = df_2s_buy.groupby(['session', 'Subject'])[['pbhh', 'pbll', 'pbhb', 'pblb']].mean()

byroun_sel_1s = df_1s_sel.groupby('Period')[['pshh', 'pshl']].mean().reset_index()
byroun_sel_2s = df_2s_sel.groupby('Period')[['pshh', 'pshl']].mean().reset_index()
byroun_buy_1s = df_1s_buy.groupby('Period')[['pbhh', 'pbll']].mean().reset_index()
byroun_buy_2s = df_2s_buy.groupby('Period')[['pbhh', 'pbll', 'pbhb', 'pblb']].mean().reset_index()

bysess_sel_1s = df_1s_sel.groupby('session')[['pshh', 'pshl']].mean()
bysess_sel_2s = df_2s_sel.groupby('session')[['pshh', 'pshl']].mean()
bysess_sel_1s['pslh'] = 1 - bysess_sel_1s['pshh']
bysess_sel_1s['psll'] = 1 - bysess_sel_1s['pshl']
bysess_sel_2s['pslh'] = 1 - bysess_sel_2s['pshh']
bysess_sel_2s['psll'] = 1 - bysess_sel_2s['pshl']

bysess_buy_1s = df_1s_buy.groupby('session')[['pbhh', 'pbll']].mean()
bysess_buy_2s = df_2s_buy.groupby('session')[['pbhh', 'pbll', 'pbhb', 'pblb']].mean()
bysess_buy_1s['pbdh'] = 1 - bysess_buy_1s['pbhh']
bysess_buy_1s['pbdl'] = 1 - bysess_buy_1s['pbll']
bysess_buy_2s['pbdh'] = 1 - bysess_buy_2s['pbhh']
bysess_buy_2s['pbdl'] = 1 - bysess_buy_2s['pbll']
bysess_buy_2s['pbdb'] = 1 - bysess_buy_2s['pbhb'] - bysess_buy_2s['pblb']

bysess_all_1s = pd.concat([bysess_buy_1s[['pbhh', 'pbdh', 'pbll', 'pbdl']], 
                           bysess_sel_1s[['pshh', 'pslh', 'pshl', 'psll']]], axis=1)
bysess_all_2s = pd.concat([bysess_buy_2s[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb']],
                           bysess_sel_2s[['pshh', 'pslh', 'pshl', 'psll']]], axis=1)

# Aggregating data for just first treatments
df_1s_sel_first = df_1s_first[df_1s_first['is_seller'] == 1]
df_2s_sel_first = df_2s_first[df_2s_first['is_seller'] == 1]
df_1s_buy_first = df_1s_first[df_1s_first['is_seller'] == 0]
df_2s_buy_first = df_2s_first[df_2s_first['is_seller'] == 0]

bysubj_sel_1s_first = df_1s_sel_first.groupby(['session', 'Subject'])[['pshh', 'pshl']].mean()
bysubj_sel_2s_first = df_2s_sel_first.groupby(['session', 'Subject'])[['pshh', 'pshl']].mean()
bysubj_buy_1s_first = df_1s_buy_first.groupby(['session', 'Subject'])[['pbhh', 'pbll']].mean()
bysubj_buy_2s_first = df_2s_buy_first.groupby(['session', 'Subject'])[['pbhh', 'pbll', 'pbhb', 'pblb']].mean()

byroun_sel_1s_first = df_1s_sel_first.groupby('Period')[['pshh', 'pshl']].mean().reset_index()
byroun_sel_2s_first = df_2s_sel_first.groupby('Period')[['pshh', 'pshl']].mean().reset_index()
byroun_buy_1s_first = df_1s_buy_first.groupby('Period')[['pbhh', 'pbll']].mean().reset_index()
byroun_buy_2s_first = df_2s_buy_first.groupby('Period')[['pbhh', 'pbll', 'pbhb', 'pblb']].mean().reset_index()

bysess_sel_1s_first = df_1s_sel_first.groupby('session')[['pshh', 'pshl']].mean()
bysess_sel_2s_first = df_2s_sel_first.groupby('session')[['pshh', 'pshl']].mean()
bysess_sel_1s_first['pslh'] = 1 - bysess_sel_1s_first['pshh']
bysess_sel_1s_first['psll'] = 1 - bysess_sel_1s_first['pshl']
bysess_sel_2s_first['pslh'] = 1 - bysess_sel_2s_first['pshh']
bysess_sel_2s_first['psll'] = 1 - bysess_sel_2s_first['pshl']

bysess_buy_1s_first = df_1s_buy_first.groupby('session')[['pbhh', 'pbll']].mean()
bysess_buy_2s_first = df_2s_buy_first.groupby('session')[['pbhh', 'pbll', 'pbhb', 'pblb']].mean()
bysess_buy_1s_first['pbdh'] = 1 - bysess_buy_1s_first['pbhh']
bysess_buy_1s_first['pbdl'] = 1 - bysess_buy_1s_first['pbll']
bysess_buy_2s_first['pbdh'] = 1 - bysess_buy_2s_first['pbhh']
bysess_buy_2s_first['pbdl'] = 1 - bysess_buy_2s_first['pbll']
bysess_buy_2s_first['pbdb'] = 1 - bysess_buy_2s_first['pbhb'] - bysess_buy_2s_first['pblb']

bysess_all_1s_first = pd.concat([bysess_buy_1s_first[['pbhh', 'pbdh', 'pbll', 'pbdl']], 
                                 bysess_sel_1s_first[['pshh', 'pslh', 'pshl', 'psll']]], axis=1)
bysess_all_2s_first = pd.concat([bysess_buy_2s_first[['pbhh', 'pbdh', 'pbll', 'pbdl', 'pbhb', 'pblb', 'pbdb']],
                                 bysess_sel_2s_first[['pshh', 'pslh', 'pshl', 'psll']]], axis=1)

for df in [byroun_sel_1s, byroun_sel_2s, bysess_sel_1s, bysess_sel_2s, byroun_sel_1s_first, byroun_sel_2s_first, bysess_sel_1s_first, bysess_sel_2s_first]:
    df['muH'] = df['pshh'] / (df['pshh'] + df['pshl'])
    df['muL'] = (1 - df['pshh']) / (2 - df['pshh'] - df['pshl'])
    df['pr_ph'] = 0.5 * df['pshh'] + 0.5 * df['pshl']
    maxH = pd.concat([df['muH'], 1 - df['muH']], axis=1).max(axis=1)
    maxL = pd.concat([df['muL'], 1 - df['muL']], axis=1).max(axis=1)
    df['inf'] = df['pr_ph'] * maxH + (1 - df['pr_ph']) * maxL

print('Saving aggregated data...')
bysubj_sel_1s.to_excel(cln_data_path + 'bysubj_sel_1s.xlsx', index=False)
bysubj_sel_2s.to_excel(cln_data_path + 'bysubj_sel_2s.xlsx', index=False)
byroun_sel_1s.to_excel(cln_data_path + 'byroun_sel_1s.xlsx', index=False)
byroun_sel_2s.to_excel(cln_data_path + 'byroun_sel_2s.xlsx', index=False)
bysess_sel_1s.to_excel(cln_data_path + 'bysess_sel_1s.xlsx', index=False)
bysess_sel_2s.to_excel(cln_data_path + 'bysess_sel_2s.xlsx', index=False)
bysubj_buy_1s.to_excel(cln_data_path + 'bysubj_buy_1s.xlsx', index=False)
bysubj_buy_2s.to_excel(cln_data_path + 'bysubj_buy_2s.xlsx', index=False)
byroun_buy_1s.to_excel(cln_data_path + 'byroun_buy_1s.xlsx', index=False)
byroun_buy_2s.to_excel(cln_data_path + 'byroun_buy_2s.xlsx', index=False)
bysess_buy_1s.to_excel(cln_data_path + 'bysess_buy_1s.xlsx', index=False)
bysess_buy_2s.to_excel(cln_data_path + 'bysess_buy_2s.xlsx', index=False)
bysess_all_1s.to_excel(cln_data_path + 'bysess_all_1s.xlsx', index=False)
bysess_all_2s.to_excel(cln_data_path + 'bysess_all_2s.xlsx', index=False)

bysubj_sel_1s_first.to_excel(cln_data_path + 'bysubj_sel_1s_first.xlsx', index=False)
bysubj_sel_2s_first.to_excel(cln_data_path + 'bysubj_sel_2s_first.xlsx', index=False)
byroun_sel_1s_first.to_excel(cln_data_path + 'byroun_sel_1s_first.xlsx', index=False)
byroun_sel_2s_first.to_excel(cln_data_path + 'byroun_sel_2s_first.xlsx', index=False)
bysess_sel_1s_first.to_excel(cln_data_path + 'bysess_sel_1s_first.xlsx', index=False)
bysess_sel_2s_first.to_excel(cln_data_path + 'bysess_sel_2s_first.xlsx', index=False)
bysubj_buy_1s_first.to_excel(cln_data_path + 'bysubj_buy_1s_first.xlsx', index=False)
bysubj_buy_2s_first.to_excel(cln_data_path + 'bysubj_buy_2s_first.xlsx', index=False)
byroun_buy_1s_first.to_excel(cln_data_path + 'byroun_buy_1s_first.xlsx', index=False)
byroun_buy_2s_first.to_excel(cln_data_path + 'byroun_buy_2s_first.xlsx', index=False)
bysess_buy_1s_first.to_excel(cln_data_path + 'bysess_buy_1s_first.xlsx', index=False)
bysess_buy_2s_first.to_excel(cln_data_path + 'bysess_buy_2s_first.xlsx', index=False)
bysess_all_1s_first.to_excel(cln_data_path + 'bysess_all_1s_first.xlsx', index=False)
bysess_all_2s_first.to_excel(cln_data_path + 'bysess_all_2s_first.xlsx', index=False)
print('Done.\n')


Aggregating by subject, round, and session...
Saving aggregated data...
Done.

